### Computation of "Isotropicness" of the Residual Noise



In [4]:
# Calculation of the residual noise in the STFT domain
import torch
import torchaudio
import soundfile as sf
import numpy as np
from DCNN.feature_extractors import Stft, IStft
win_len = 400
win_inc = 100
fft_len = 512
fbins = int(fft_len/2 + 1)
fs = 16000

stft = Stft(n_dft=fft_len, hop_size=win_inc, win_length=win_len)
istft = IStft(n_dft=fft_len, hop_size=win_inc, win_length=win_len)


 ### Compute individual gains for from the enhanced and noisy signals in the STFT domain

In [5]:
sig_noisy,_ = torchaudio.load('/Users/vtokala/Documents/Research/Databases/Dataset_Binaural_2S/ICASSP_Testset/isoNoiseCoh/Noisy_input/p278_056_1_0dB.wav')
sig_enhanced_snr,_ = torchaudio.load('/Users/vtokala/Documents/Research/Databases/Dataset_Binaural_2S/ICASSP_Testset/isoNoiseCoh/p278_056_1_SNR.wav')
sig_enhanced_pl,_ = torchaudio.load('/Users/vtokala/Documents/Research/Databases/Dataset_Binaural_2S/ICASSP_Testset/isoNoiseCoh/p278_056_1_SNR_PL.wav')
iso_noise,_ = torchaudio.load('/Users/vtokala/Documents/Research/Databases/Dataset_Binaural_2S/ICASSP_Testset/isoNoiseCoh/isoNoise.wav')
sig_enhanced_bt,_ = torchaudio.load('/Users/vtokala/Documents/Research/Databases/Dataset_Binaural_2S/ICASSP_Testset/isoNoiseCoh/p278_056_1_BT.wav')
sig_enhanced_bsobm,srb = torchaudio.load('/Users/vtokala/Documents/Research/Databases/Dataset_Binaural_2S/ICASSP_Testset/isoNoiseCoh/p278_056_1_bsobm.wav')
sig_noisy_bsobm,_ = torchaudio.load('/Users/vtokala/Documents/Research/Databases/Dataset_Binaural_2S/ICASSP_Testset/isoNoiseCoh/p278_056_1_noisy_bsobm.wav')

In [6]:
S_hat_l_pl = stft(sig_enhanced_pl[0,:])
S_hat_r_pl = stft(sig_enhanced_pl[1,:])

S_hat_l_bsobm = stft(sig_enhanced_bsobm[0,:])
S_hat_r_bsobm = stft(sig_enhanced_bsobm[1,:])

X_l = stft(sig_noisy[0,:])
X_r = stft(sig_noisy[1,:])

X_l_bsobm = stft(sig_noisy_bsobm[0,:])
X_r_bsobm = stft(sig_noisy_bsobm[1,:])

S_hat_l_snr = stft(sig_enhanced_snr[0,:])
S_hat_r_snr = stft(sig_enhanced_snr[1,:])

S_hat_l_bt = stft(sig_enhanced_bt[0,:])
S_hat_r_bt = stft(sig_enhanced_bt[1,:])

V_l = stft(iso_noise[0,:])
V_r = stft(iso_noise[1,:])

G_l_pl = S_hat_l_pl / X_l # Gain applied the signal for enhancement - typically the CRM output of the network
G_r_pl = S_hat_r_pl / X_r

G_l_snr = S_hat_l_snr / X_l # Gain applied the signal for enhancement - typically the CRM output of the network
G_r_snr = S_hat_r_snr / X_r 

G_l_bt = S_hat_l_bt / X_l
G_r_bt = S_hat_r_bt / X_r

# G_l_bsobm = S_hat_l_bsobm / X_l_bsobm
# G_r_bsobm = S_hat_r_bsobm / X_r_bsobm

V_hat_l_pl = G_l_pl * V_l
V_hat_r_pl = G_r_pl * V_r


V_hat_l_snr = G_l_snr * V_l
V_hat_r_snr = G_r_snr * V_r

V_hat_l_bt = G_l_bt * V_l
V_hat_r_bt = G_r_bt * V_r

# V_hat_l_bsobm = G_l_bsobm * V_l
# V_hat_r_bsobm = G_r_bsobm * V_r






In [7]:

S_hat_l_pl.shape

torch.Size([257, 320])

In [8]:
v_hat_l_pl = istft(V_hat_l_pl)
v_hat_r_pl = istft(V_hat_r_pl)

v_hat_l_snr = istft(V_hat_l_snr)
v_hat_r_snr = istft(V_hat_r_snr)

v_hat_l_bt = istft(V_hat_l_bt)
v_hat_r_bt = istft(V_hat_r_bt)

# v_hat_l_bsobm = istft(V_hat_l_bsobm)
# v_hat_r_bsobm = istft(V_hat_r_bsobm)

v_hat_pl = torch.stack([v_hat_l_pl, v_hat_r_pl], dim=1)
v_hat_snr = torch.stack([v_hat_l_snr, v_hat_r_snr], dim=1)
v_hat_bt = torch.stack([v_hat_l_bt, v_hat_r_bt], dim=1)
# v_hat_bsobm = torch.stack([v_hat_l_bsobm,v_hat_r_bsobm],dim=1)


DATASET_PATH = '/Users/vtokala/Documents/Research/Databases/Dataset_Binaural_2S/ICASSP_Testset/isoNoiseCoh/'



In [9]:
v_hat_bt.shape

torch.Size([31900, 2])

In [10]:
sf.write(DATASET_PATH + "resNoise_pl.wav", v_hat_pl.numpy(), 16000)
sf.write(DATASET_PATH + "resNoise_snr.wav", v_hat_snr.numpy(), 16000)
sf.write(DATASET_PATH + "resNoise_bt.wav", v_hat_bt.numpy(), 16000)
# sf.write(DATASET_PATH + "resNoise_bsobm.wav", v_hat_bsobm.numpy(), 10000)